In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-124637")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-124637
Azure region: southcentralus
Subscription id: 55e71b9d-a209-42c0-8818-ca9cc885909c
Resource group: aml-quickstarts-124637


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_config = AmlCompute.provisioning_configuration(vm_size= "Standard_D2_V2",max_nodes=4)
cpu_cluster = ComputeTarget.create(ws,provisioning_configuration = compute_config,name= "udacity-Project1")

In [4]:
pwd

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/project/code/Users/odl_user_124637'

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    parameter_space={
        "learning_rate": uniform(0.05,0.1)
        })

# Specify a Policy
policy = BanditPolicy(slack_factor= 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.', compute_target = cpu_cluster,entry_script= 'train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling= ps, policy=policy,primary_metric_name='accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=1)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
# from azureml.widgets import RunDetails
# HyperDriveRun(exp,hyperdrive_config= hyperdrive_config)
hyper_run = exp.submit(hyperdrive_config)
RunDetails(hyper_run).show()
# hyper_run.wait_for_completion()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
import joblib
# Get your best run and save the model from that run.
best_run = hyper_run.get_best_run_by_primary_metric()
joblib.dump(best_run,'saved_model')


['saved_model']

In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv",separator=',')

In [32]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

In [33]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='mean_squared_error',
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [37]:
# Submit your automl run
from azureml.train.automl.run import AutoMLRun
from azureml.core.experiment import Experiment
### YOUR CODE HERE ###
# ws = Workspace.from_config()
experiment = Experiment(ws, "automl_test_experiment")
# automl_run = AutoMLRun(experiment)
run = experiment.submit(config= automl_config, show_output=True)

ConfigException: ConfigException:
	Message: Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "label_column_name",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        },
        "reference_code": "061ed905-e59e-42b9-ad95-cd18f40b4358"
    }
}

In [ ]:
# Retrieve and save your best automl model.
best = automl_run.get_best_child()
### YOUR CODE HERE ###
joblib.dump(automl_saved_model)